###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import tensorflow as tf
from aircraft_models import AirBinaryTemporalCNN
from tfrecord_dataset import feature_description

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
NUMBER_MFCC = 128
BUFFER_SIZE = 1000
BATCH_SIZE = 128

In [3]:
# Dataset location
train_file = 'exports/2020-02-03 11-55-25/train.tfrecord'
test_file = 'exports/2020-02-03 11-55-25/test.tfrecord'

In [4]:
# Parses observation from proto format
def parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    observation['spec'] = tf.reshape(observation['spec'],(NUMBER_MFCC,-1))
    return observation
    
# Converts into correct format for training (input,output) = (spec,label)
def transform_observation(data):
    return tf.transpose(data['spec']),data['label'] == b'Airbus'

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(parse_observation,num_parallel_calls=AUTOTUNE)
observation = next(iter(train_dataset))
_,time = observation['spec'].shape
train_dataset = train_dataset.map(transform_observation,num_parallel_calls=AUTOTUNE).cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(parse_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(transform_observation,num_parallel_calls=AUTOTUNE).cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Building dataset and model
builder = AirBinaryTemporalCNN()
model = builder.build_model((time,NUMBER_MFCC))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 401, 128)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 51, 16)            32784     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 16)            784       
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 33,585
Trainable params: 33,585
Non-trainable params: 0
_________________________________________________________

In [8]:
# Training model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset,epochs=100,validation_data=test_dataset)

Epoch 1/100
7/7 [==============================] - 3s 411ms/step - loss: 1.2642 - accuracy: 0.5653 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
7/7 [==============================] - 0s 40ms/step - loss: 1.0904 - accuracy: 0.6836 - val_loss: 1.3240 - val_accuracy: 0.6250
Epoch 3/100
7/7 [==============================] - 0s 40ms/step - loss: 0.9996 - accuracy: 0.7511 - val_loss: 1.4610 - val_accuracy: 0.6250
Epoch 4/100
7/7 [==============================] - 0s 39ms/step - loss: 0.9640 - accuracy: 0.7928 - val_loss: 1.3491 - val_accuracy: 0.6250
Epoch 5/100
7/7 [==============================] - 0s 42ms/step - loss: 0.8949 - accuracy: 0.8457 - val_loss: 1.3968 - val_accuracy: 0.6250
Epoch 6/100
7/7 [==============================] - 0s 40ms/step - loss: 0.8550 - accuracy: 0.8694 - val_loss: 1.2579 - val_accuracy: 0.6562
Epoch 7/100
7/7 [==============================] - 0s 40ms/step - loss: 0.8194 - accuracy: 0.8739 - val_loss: 1.2842 - val_accuracy: 0.6562
Epoch 8/100

Epoch 60/100
7/7 [==============================] - 0s 40ms/step - loss: 0.1308 - accuracy: 0.9865 - val_loss: 0.8874 - val_accuracy: 0.8750
Epoch 61/100
7/7 [==============================] - 0s 40ms/step - loss: 0.1238 - accuracy: 0.9865 - val_loss: 0.9273 - val_accuracy: 0.8750
Epoch 62/100
7/7 [==============================] - 0s 40ms/step - loss: 0.1257 - accuracy: 0.9854 - val_loss: 0.9626 - val_accuracy: 0.8750
Epoch 63/100
7/7 [==============================] - 0s 41ms/step - loss: 0.1219 - accuracy: 0.9865 - val_loss: 0.8869 - val_accuracy: 0.9062
Epoch 64/100
7/7 [==============================] - 0s 40ms/step - loss: 0.1232 - accuracy: 0.9854 - val_loss: 0.8277 - val_accuracy: 0.9062
Epoch 65/100
7/7 [==============================] - 0s 43ms/step - loss: 0.1183 - accuracy: 0.9842 - val_loss: 1.2506 - val_accuracy: 0.8750
Epoch 66/100
7/7 [==============================] - 0s 40ms/step - loss: 0.1152 - accuracy: 0.9876 - val_loss: 0.9265 - val_accuracy: 0.9062
Epoch 67/100
